In [ ]:
!pip install pyvis sparqlwrapper

In [ ]:
rdf_endpoint = ''

In [ ]:
# Setup, run once
from pyvis.network import Network
from SPARQLWrapper import SPARQLWrapper, JSON

def runQuery(myQuery):
    endpoint = SPARQLWrapper(rdf_endpoint)
    endpoint.setQuery(myQuery)
    endpoint.method = "POST"
    endpoint.setReturnFormat(JSON)
    return endpoint.query().convert()["results"]["bindings"]

## Get classes present in the database

In [ ]:
graph = Network(notebook=True, directed=True, width=900, height=720)
graph.show_buttons(filter_=['physics'])
graph.barnes_hut(
    gravity=-3050,
    spring_length=120,
    overlap=0.2
)

classResults = runQuery("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?class
WHERE {
    ?instance rdf:type ?class.
}
""")

classes = []
for classResult in classResults:
    #if it is a bnode, then skip
    if classResult["class"]["type"] == "bnode":
        continue
    
    myClass = classResult["class"]["value"]
    
    # Ignore ontology nodes
    if not (myClass.startswith("http://www.w3.org/1999/02/22-rdf-syntax-ns#") | 
                    myClass.startswith("http://www.w3.org/2002/07/owl#") | 
                    myClass.startswith("http://www.w3.org/2000/01/rdf-schema#")):
        graph.add_node(myClass)
        classes.append(myClass)

In [ ]:
graph.show('classes.html')

## Extract the edges between the instances of these classes and use them to connect them

In [ ]:
def getRelationsForClass(classUri):
    myQuery = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

        SELECT DISTINCT ?p ?o
        WHERE {
            ?instances rdf:type <%s>.
            ?instances ?p ?relatedClass.
            ?relatedClass rdf:type ?o.
        }
    """ % classUri
    return runQuery(myQuery)

for myClass in classes:
    relations = getRelationsForClass(myClass)
    for relation in relations:
        # predicateUri = str(relation[0])
        predicateUri = relation["p"]["value"]
        domainUri = myClass
        # rangeUri = str(relation[1])
        rangeUri = relation["o"]["value"]

        # Ignore ontology edges
        if not (predicateUri.startswith("http://www.w3.org/1999/02/22-rdf-syntax-ns#") | 
                predicateUri.startswith("http://www.w3.org/2002/07/owl#") | 
                predicateUri.startswith("http://www.w3.org/2000/01/rdf-schema#")):

            graph.add_edge(domainUri, rangeUri, title=predicateUri, uri=predicateUri)

In [ ]:
graph.show('connected.html')

## Extract namespaces to make the URIs more readable

In [ ]:
import re

namespaces = set()

# Extract all the namespaces by removing everything after the last / or #
for uri in [node['id'] for node in graph.nodes] + [edge['title'] for edge in graph.edges]:
    namespace = re.sub(r'(.+[#/])[^#/]+', r'\1', uri)
    namespaces.add(namespace)

# Take the last part of the URI to be the prefix
# (probably too naive to generalize well, could build in checks)
namespaces = {ns: re.split(r'[#/]', ns)[-2] for ns in namespaces}

for ns, pref in namespaces.items():
    print(f'{ns} - {pref}')

In [ ]:
# Transform URIs using prefixes
for node in graph.nodes:
    for ns, pref in namespaces.items():
        if node['id'].startswith(ns):
            node['label'] = node['id'].replace(ns, pref + ':')

for edge in graph.edges:
    for ns, pref in namespaces.items():
        if edge['title'].startswith(ns):
            edge['title'] = edge['title'].replace(ns, pref + ':')

In [ ]:
graph.show('prefixed.html')

## Get labels to make things actually human-readable

In [ ]:
def getLabelForUri(uri):
    myQuery = """
        prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?uriLabel
        WHERE {
            <%s> rdfs:label ?uriLabel.
        }""" % str(uri)

    results = runQuery(myQuery)
    
    for result in results:
        retVal = str(result["uriLabel"]["value"])
        return retVal
    return ''

for node in graph.nodes:
    newLabel = getLabelForUri(node['id'])
    if newLabel:
        node['label'] = f'{newLabel}\n{node["label"]}'

prev_edges = {}
for edge in graph.edges:
    if edge['uri'] not in prev_edges:
        newLabel = getLabelForUri(edge['uri'])
        prev_edges[edge['uri']] = newLabel
        if newLabel:
            edge['title'] = f'{newLabel}\n{edge["title"]}'
    else:
        if prev_edges[edge['uri']]:
            edge['title'] = f'{prev_edges[edge["uri"]]}\n{edge["title"]}'


In [ ]:
graph.show('labelled.html')